In [1]:
!env | grep VIRTUAL_ENV

VIRTUAL_ENV=/home/alibek/pix2pix_TecoGAN/venv


In [2]:
### IMPORTING NECESSARY LIRARIES
import cv2
import tensorflow as tf
import numpy as np
import torch
import pyfakewebcam
from threading import Thread

import yaml
from TecoGAN_PyTorch.codes.utils import base_utils
import os.path as osp
from TecoGAN_PyTorch.codes.models import define_model
from imutils.video import FPS

In [3]:
### TENSORFLOW GPU CONFIGURATIONS
# tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
    # tf.config.experimental.set_memory_growth(gpus[1], True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU


In [4]:
### PYTORCH GPU CONFIGURATIONS
device = torch.device('cuda:1')
torch.cuda.set_device(device)
print('Active CUDA Device: GPU', torch.cuda.current_device())

Active CUDA Device: GPU 1


In [5]:
### PIX2PIX AND TECOGAN CONFIGURATIONS
SIZE = 512
NORM = 255.5

In [6]:
### LOAD PIX2PIX MODEL
generator = tf.saved_model.load("./model/pix2pixTF512")

In [7]:
### USE OPENCV WITH MULTITHREADING
class WebcamVideoStream:
    def __init__(self, src=0, device=None):
        # initialize the video camera stream and read the first frame
        # from the stream
        self.stream = cv2.VideoCapture(src, device)
        (self.grabbed, self.frame) = self.stream.read()
        # initialize the variable used to indicate if the thread should
        # be stopped
        self.stopped = False

    def start(self):
        # start the thread to read frames from the video stream
        Thread(target=self.update, args=()).start()
        return self

    def update(self):
        # keep looping infinitely until the thread is stopped
        while True:
            # if the thread indicator variable is set, stop the thread
            if self.stopped:
                return

            # otherwise, read the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        # return the frame most recently read
        return self.frame

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True

In [8]:
### TECOGAN CONFIGURATIONS
with open(osp.join("./TecoGAN_PyTorch/experiments_BD/TecoGAN/001", "live.yml"), 'r') as f:
    opt = yaml.load(f.read(), Loader=yaml.FullLoader)

# ----------------- general configs ----------------- #
# experiment dir
opt['exp_dir'] = "./TecoGAN-PyTorch/experiments_BD/TecoGAN/001"

# random seed
base_utils.setup_random_seed(opt['manual_seed'])

# logger
base_utils.setup_logger('base')
# opt['verbose'] = opt.get('verbose', False)
opt['verbose'] = False

# device
opt['device'] = device

# setup paths
base_utils.setup_paths(opt, mode='test')

# run
opt['is_train'] = False



In [9]:
### FUNCTION FOR USING PIX2PIX
def pix2pix(img):
    # print(img.shape)
    # print(type(img))
    input_image = tf.cast(img, tf.float32)
    input_image = tf.image.resize(input_image, [SIZE, SIZE],
                                    method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    input_image = (input_image / NORM) - 1
    ext_image = tf.expand_dims(input_image, axis=0)
    prediction = generator(ext_image, training=True)
    # pil_image = tf.keras.preprocessing.image.array_to_img(prediction[0])
    # print(pil_image.shape)
    # print(type(pil_image))
    return prediction[0].numpy()

# def get_from_model(iq, oq):
#     # print(style.RED + str(oq.maxsize))
#     while not exitFlag:
#         if not iq.empty():
#             image = iq.get()
#             # print(image.shape)
#             # print(type(image))
#             input_image = tf.cast(image, tf.float32)
#             input_image = tf.image.resize(input_image, [SIZE, SIZE],
#                                           method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#             input_image = (input_image / NORM) - 1
#             # input_image = load_from_video(image)
#             ext_image = tf.expand_dims(input_image, axis=0)
#             prediction = generator(ext_image, training=True)
#             # generated_image = generate_images(generator, ext_image)
#             # pil_image = tf.keras.preprocessing.image.array_to_img(
#             #     generated_image)
#             pil_image = tf.keras.preprocessing.image.array_to_img(
#                 prediction[0])
#             if not oq.full():
#                 oq.put(np.array(pil_image))

In [10]:
### FONCTION FOR DENOISING WITH BILETAREL
def denoising(img):
    img = cv2.bilateralFilter(img,15,80,80)
    return img

In [11]:
### FUNCTION FOR WEBCAM
camera = pyfakewebcam.FakeWebcam('/dev/video0', SIZE*4, SIZE*4)
def send_to_webcam(img):
    camera.schedule_frame(img)

In [12]:
### FUNCTION FOR SHOWING IMAGES
def showImages(name, img):
    if not tf.is_tensor(img):
        cv2.imshow(name, img)
    else:
        cv2.imshow(name, img.numpy())

In [13]:
### FUNCTION FOR USING TECOGAN
def live(opt):
    # cv2.namedWindow("pix2pix image", cv2.WINDOW_NORMAL)
    # cv2.namedWindow("tecogan image", cv2.WINDOW_NORMAL)
    cv2.namedWindow("denoise image", cv2.WINDOW_NORMAL)
    # cap = cv2.VideoCapture(0)
    cap = WebcamVideoStream(src=2).start()

    # logging
    logger = base_utils.get_logger('base')
    if opt['verbose']:
        logger.info('{} Configurations {}'.format('=' * 20, '=' * 20))
        base_utils.print_options(opt, logger)

    # infer and evaluate performance for each model
    for load_path in opt['model']['generator']['load_path_lst']:
        # setup model index
        model_idx = osp.splitext(osp.split(load_path)[-1])[0]

        # log
        logger.info('=' * 40)
        logger.info('Testing model: {}'.format(model_idx))
        logger.info('=' * 40)

        # create model
        opt['model']['generator']['load_path'] = load_path
        model = define_model(opt)
        fps = FPS().start()
        while True:
            image = cap.read()
            if image.any():
                # print(image.shape)
                # image = cv2.resize(image, (128, 128))
                pix2pix_image = pix2pix(image)
                # showImages("pix2pix image", pix2pix_image)
                # cv2.imshow("pix2pix image", pix2pix_image)
                norm_image = cv2.normalize(
                    pix2pix_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                # torch.cuda.set_device(device)
                # print('Active CUDA Device: GPU', torch.cuda.current_device())
                tmp_torch = torch.from_numpy(norm_image[None, :, :, :]).cuda(1)
                hr_image = model.infer_live(tmp_torch)
                tecogan_image = hr_image[0]
                # showImages("tecogan image", tecogan_image)
                # cv2.imshow("tecogan image", tecogan_image)
                # norm_image = cv2.normalize(
                #     tecogan_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                # tecogan_image = tf.image.resize(norm_image, (1024, 1024))
                tecogan_image = cv2.resize(tecogan_image, (1024, 1024))
                denoise_image = denoising(tecogan_image)
                showImages("denoise image", denoise_image)
                # cv2.imshow("denoise image", denoise_image)
                # send_to_webcam(denoise_image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    cap.stop()
                    break
                fps.update()
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    # logging
    logger.info('Finish testing')
    logger.info('=' * 40)

In [14]:
### testing area
live(opt)

2021-04-27 17:52:50,384 [INFO]: ========================================
2021-04-27 17:52:50,385 [INFO]: Testing model: TecoGAN_BD_iter500000
2021-04-27 17:52:50,385 [INFO]: ========================================
2021-04-27 17:53:10,328 [INFO]: Finish testing
2021-04-27 17:53:10,329 [INFO]: ========================================
[INFO] elasped time: 16.05
[INFO] approx. FPS: 3.99
